In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils.streams import populate_streams

%matplotlib inline
plt.style.use('ggplot')


In [ ]:
stream_root_folder = r"C:\Users\neurogears\Desktop\EC_datasets\BenchmarkRun2022_09_22_10_30_02"

streams = populate_streams(root = stream_root_folder, autoload=False)